## prod and slicing in gurobipy and ticdat

I'm playing around with adding a `prod` to `ticdat`. It will only work with the `gurobipy.tupledict` object, so this is an engine biased function.

What's interesting is that because you define the field names for `ticdat`, you can slice by name instead of by position. This really improves readability in my opinion. 

Have you guys ever discussed named slicing? It would probably be an easy extention to `gurobipy.tupledict` to include field (aka position) names when calling `addVars`.

In [1]:
import gurobipy as gu
from ticdat import TicDatFactory

In [2]:
mdl = gu.Model()
temp_vars = mdl.addVars([(1,2), ("a","b")], vtype=gu.GRB.BINARY,name='temp_vars')
temp_vars

{(1, 2): <gurobi.Var temp_vars[1,2]>, ('a', 'b'): <gurobi.Var temp_vars[a,b]>}

In [3]:
tdf = TicDatFactory(table = [["Field One", "Field Two"],["Data Field"]])
dat = tdf.TicDat(table = {(1,2):12, ("a","b"):2})
dat.table

{(1, 2): _td:{'Data Field': 12}, ('a', 'b'): _td:{'Data Field': 2}}

Here is positional slicing with `gurobipy.prod`. For `ticdat` to play along, it needs collapse its inner dicts to actual data values with the `field_dict` function.

In [4]:
temp_vars.prod(dat.table.field_dict(), 'a', '*')

<gurobi.LinExpr: 2.0 temp_vars[a,b]>

Here is named slicing. It renders the same result, but it's more self-documenting. It's more explicit than positional slicing in indicating that the product is over those tuples whose "field_one" value is 'a'.

In [5]:
dat.table.prod(temp_vars, field_one = "a")

<gurobi.LinExpr: 2.0 temp_vars[a,b]>

Let's see it again - first positional slicing.

In [6]:
temp_vars.prod(dat.table.field_dict(), '*', 2)

<gurobi.LinExpr: 12.0 temp_vars[1,2]>

Then named slicing.

In [7]:
dat.table.prod(temp_vars, field_two = 2)

<gurobi.LinExpr: 12.0 temp_vars[1,2]>

Just to include the basic case, we verify that both `ticdat` and `gurobipy` implement the same `prod`.

In [8]:
temp_vars.prod(dat.table.field_dict())

<gurobi.LinExpr: 12.0 temp_vars[1,2] + 2.0 temp_vars[a,b]>

In [9]:
dat.table.prod(temp_vars)

<gurobi.LinExpr: 12.0 temp_vars[1,2] + 2.0 temp_vars[a,b]>

# the super duper section

In [10]:
super_vars = mdl.addVars([(1,i) for i in range(3,10)] +  
                         [("a",i) for i in range(5,12)] + 
                         [(2,i) for i in range(2,9)], 
                         vtype=gu.GRB.BINARY,name='super_vars')
mdl.update()
super_vars

{(1, 3): <gurobi.Var super_vars[1,3]>,
 (1, 4): <gurobi.Var super_vars[1,4]>,
 (1, 5): <gurobi.Var super_vars[1,5]>,
 (1, 6): <gurobi.Var super_vars[1,6]>,
 (1, 7): <gurobi.Var super_vars[1,7]>,
 (1, 8): <gurobi.Var super_vars[1,8]>,
 (1, 9): <gurobi.Var super_vars[1,9]>,
 (2, 2): <gurobi.Var super_vars[2,2]>,
 (2, 3): <gurobi.Var super_vars[2,3]>,
 (2, 4): <gurobi.Var super_vars[2,4]>,
 (2, 5): <gurobi.Var super_vars[2,5]>,
 (2, 6): <gurobi.Var super_vars[2,6]>,
 (2, 7): <gurobi.Var super_vars[2,7]>,
 (2, 8): <gurobi.Var super_vars[2,8]>,
 ('a', 5): <gurobi.Var super_vars[a,5]>,
 ('a', 6): <gurobi.Var super_vars[a,6]>,
 ('a', 7): <gurobi.Var super_vars[a,7]>,
 ('a', 8): <gurobi.Var super_vars[a,8]>,
 ('a', 9): <gurobi.Var super_vars[a,9]>,
 ('a', 10): <gurobi.Var super_vars[a,10]>,
 ('a', 11): <gurobi.Var super_vars[a,11]>}

In [11]:
from ticdat import Slicer
slicer = Slicer([(1,i) for i in range(6,9)] +  
                [("a",i) for i in range(7,10)] + 
                [(2,i) for i in range(2,9)], field_names = ["boger", "woger"])

## Remember - goal is to avoid any slice variables (i.e. no comprehsions) 

Can use an anonymous function to determine the coefficient for each entry in the created linear expression.

In [12]:
slicer.select(boger = 1).prod(super_vars, scalar = lambda b,w : w*100)

<gurobi.LinExpr: 0.0 super_vars[1,3] + 0.0 super_vars[1,4] + 0.0 super_vars[1,5] + 600.0 super_vars[1,6] + 700.0 super_vars[1,7] + 800.0 super_vars[1,8] + 0.0 super_vars[1,9]>

Also can use dict for the same purpose. (Default scalar is 1).

In [13]:
slicer.select(woger = 8).prod(super_vars, scalar = {(1,8):10,(2,8):20} )

<gurobi.LinExpr: 10.0 super_vars[1,8] + super_vars[a,8] + 20.0 super_vars[2,8]>

Also can use an anonymous function for filtering (sidestepping another need for comprehensions). Here the filter can't truly remove anything, but it can set the coeficient to zero if the filter function returns false.

# NB 
The zero entry below shouldn't be there. My code is passing only dicts with non-zero coefs, in an effort to increase efficiency, but `gurobipy.tupledict` is stubbornly refusing to take the hint.

In [14]:
slicer.select(woger = 8).prod(super_vars, filter = lambda b,w : b in [1,2])

<gurobi.LinExpr: super_vars[1,8] + 0.0 super_vars[a,8] + super_vars[2,8]>

Not really needed, but the scalar argument recognizes a fixed value.

In [15]:
slicer.select(woger = 8).prod(super_vars, filter = lambda b,w : b in [1,2], scalar = 12 )

<gurobi.LinExpr: 12.0 super_vars[1,8] + 0.0 super_vars[a,8] + 12.0 super_vars[2,8]>

In [16]:
12 * slicer.select(woger = 8).prod(super_vars, filter = lambda b,w : b in [1,2])

<gurobi.LinExpr: 12.0 super_vars[1,8] + 0.0 super_vars[a,8] + 12.0 super_vars[2,8]>